Here we implement the boundary operator, as well as the combinatorical Laplacian.

In [148]:
import numpy as np
import itertools as it
from scipy.sparse import csr_matrix

Define some functions. 'state' is a list of tuples of zeros and ones of length n_vertices, specifying a superposition of simplices. 'vec' is a vector of length 2**n_vertices, specifying the superposition of simplices represented in the multi-quibit basis.

In [149]:
def vec_to_state(vec):
    n_vertices = int(np.log2(len(vec)))
    basis_list = list(it.product(range(2), repeat=n_vertices))
    indices = np.nonzero(vec)[0]
    return [basis_list[i] for i in indices]

def state_to_vec(state):
    ''' state is a list of state '''
    n_vertices = len(state[0])
    basis_list = list(it.product(range(2), repeat=n_vertices))
    basis_dict = {basis_list[i]: i for i in range(2**n_vertices)}
    vec = np.zeros(2**n_vertices)
    for s in state:
        vec[basis_dict[s]] = 1
    return vec

def boundary_operator(x_tuple):
    x_np = np.array(x_tuple)
    indices = np.nonzero(x_np)[0]
    dictionary = {}
    for i in range(len(indices)):
        helper = x_np.copy()
        helper[indices[i]] = 0
        dictionary[tuple(helper)] = (-1)**i
    return dictionary

def boundary_operator_dict(n_vertices):
    dictionary = {}
    dictionary[(tuple([0 for i in range(n_vertices)]),tuple([0 for i in range(n_vertices)]))] = 0
    for b in it.product(range(2), repeat=n_vertices):
        helper = boundary_operator(b)
        for key in helper.keys():
            dictionary[(tuple(b),key)] = helper[key]
    return dictionary

def boundary_operator_crsmat(n_vertices):
    dictionary = boundary_operator_dict(n_vertices)
    basis_list = list(it.product(range(2), repeat=n_vertices))
    basis_dict = {basis_list[i]: i for i in range(2**n_vertices)}
    col = np.array([basis_dict[index[0]] for index in dictionary.keys()])
    row = np.array([basis_dict[index[1]] for index in dictionary.keys()])
    data = np.array(list(dictionary.values()))
    return csr_matrix((data, (row, col)), shape=(2**n_vertices, 2**n_vertices))

Get boundary operator as sparse crs matrix

In [150]:
mat = boundary_operator_crsmat(3).toarray()
mat

array([[ 0,  1,  1,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  1,  0,  0],
       [ 0,  0,  0, -1,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0, -1, -1,  0],
       [ 0,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

Check, that $\partial^2 = 0$

In [151]:
mat@mat

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

Example boundary is calculated.

In [152]:
state = [(1,1,1)]
n_vertices = 3
mat = boundary_operator_crsmat(n_vertices).toarray()
vec = state_to_vec(state)
vec_to_state(mat@vec)

[(0, 1, 1), (1, 0, 1), (1, 1, 0)]